In [1]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [2]:
import os
path = "/content/drive/MyDrive/os/os-project/datasets"
os.chdir(path)

import pandas as pd
files = os.listdir()

files

['Cleaned dataset.csv',
 'Train.csv',
 'Test.csv',
 'Validation.csv',
 'Train_1.csv',
 'Train_2.csv',
 'Train_3.csv',
 'Train_4.csv']

In [3]:
train_1_df = pd.read_csv("Train_1.csv", engine='python')
train_2_df = pd.read_csv("Train_2.csv", engine='python')
train_3_df = pd.read_csv("Train_3.csv", engine='python')
train_4_df = pd.read_csv("Train_4.csv", engine='python')
test_df = pd.read_csv("Test.csv", engine='python')
val_df = pd.read_csv("Validation.csv", engine='python')


In [4]:
val_results = {}
results = {}

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

logistic_classifier = LogisticRegression(random_state=42)
dt_classifier = DecisionTreeClassifier(random_state=42)
svm_classifier = SVC(random_state=42)
rf_classifier = RandomForestClassifier(random_state=42)

In [6]:
import numpy as np
import torch
from sklearn.metrics import accuracy_score
from transformers import RobertaTokenizer, RobertaModel

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base').to(device)

# Tokenize the queries and convert them to input IDs for RoBERTa
train_1_inputs = tokenizer(list(train_1_df['Query']), padding=True, truncation=True, return_tensors="pt").to(device)
train_2_inputs = tokenizer(list(train_2_df['Query']), padding=True, truncation=True, return_tensors="pt").to(device)
train_3_inputs = tokenizer(list(train_3_df['Query']), padding=True, truncation=True, return_tensors="pt").to(device)
train_4_inputs = tokenizer(list(train_4_df['Query']), padding=True, truncation=True, return_tensors="pt").to(device)
test_inputs = tokenizer(list(test_df['Query']), padding=True, truncation=True, return_tensors="pt").to(device)
val_inputs = tokenizer(list(val_df['Query']), padding=True, truncation=True, return_tensors="pt").to(device)

batch_size = 128

# Initialize lists to store model outputs
train_1_outputs = []
train_2_outputs = []
train_3_outputs = []
train_4_outputs = []
test_outputs = []
val_outputs = []

# Compute model outputs in batches for training and testing data
for data, outputs in zip([train_1_inputs, train_2_inputs, train_3_inputs, train_4_inputs, test_inputs, val_inputs], [train_1_outputs, train_2_outputs, train_3_outputs, train_4_outputs, test_outputs, val_outputs]):
    for i in range(0, len(data['input_ids']), batch_size):
        with torch.no_grad():
            batch_outputs = model(**{k: v[i:i+batch_size] for k, v in data.items()})
            outputs.append(batch_outputs.last_hidden_state.mean(dim=1).cpu().numpy())


X_1_train, X_2_train, X_3_train, X_4_train, X_test, X_val = map(torch.tensor, [np.concatenate(train_1_outputs), np.concatenate(train_2_outputs), np.concatenate(train_3_outputs), np.concatenate(train_4_outputs), np.concatenate(test_outputs), np.concatenate(val_outputs)])


Device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
def train_model(model, model_name, X_train, Y_train):
  # fit in training dataset
  model.fit(X_train, Y_train.fillna(0))

 # validate the model
  val_results[model_name] = model.predict(X_val)
  accuracy = accuracy_score(val_df['Label'].fillna(0), val_results[model_name])
  print("Validation Accuracy:", accuracy)

  # fit in validation dataset
  model.fit(X_val, val_df['Label'].fillna(0))

  # test the model
  results[model_name] = model.predict(X_test)
  accuracy = accuracy_score(test_df['Label'].fillna(0), results[model_name])
  print("Test Accuracy:", accuracy)



train_model(logistic_classifier, "Roberta Logistic Regression", X_1_train, train_1_df['Label'])
train_model(dt_classifier, "Roberta Decision Tree", X_2_train, train_2_df['Label'])
train_model(svm_classifier, "Roberta SVM", X_3_train, train_3_df['Label'])
train_model(rf_classifier, "Roberta Random Forest", X_4_train, train_4_df['Label'])



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Validation Accuracy: 0.9902431564459397


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Test Accuracy: 0.9920173721556154
Validation Accuracy: 0.9454962532497324
Test Accuracy: 0.9465378027893321
Validation Accuracy: 0.9847071417647958
Test Accuracy: 0.9879190114998777
Validation Accuracy: 0.9783758984554213
Test Accuracy: 0.9811291901149988


In [8]:
import numpy as np
import torch
from sklearn.metrics import accuracy_score
from transformers import BertTokenizer, BertModel

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased').to(device)

# Tokenize the queries and convert them to input IDs for RoBERTa
train_1_inputs = tokenizer(list(train_1_df['Query']), padding=True, truncation=True, return_tensors="pt").to(device)
train_2_inputs = tokenizer(list(train_2_df['Query']), padding=True, truncation=True, return_tensors="pt").to(device)
train_3_inputs = tokenizer(list(train_3_df['Query']), padding=True, truncation=True, return_tensors="pt").to(device)
train_4_inputs = tokenizer(list(train_4_df['Query']), padding=True, truncation=True, return_tensors="pt").to(device)
test_inputs = tokenizer(list(test_df['Query']), padding=True, truncation=True, return_tensors="pt").to(device)
val_inputs = tokenizer(list(val_df['Query']), padding=True, truncation=True, return_tensors="pt").to(device)

batch_size = 128

# Initialize lists to store model outputs
train_1_outputs = []
train_2_outputs = []
train_3_outputs = []
train_4_outputs = []
test_outputs = []
val_outputs = []

# Compute model outputs in batches for training and testing data
for data, outputs in zip([train_1_inputs, train_2_inputs, train_3_inputs, train_4_inputs, test_inputs, val_inputs], [train_1_outputs, train_2_outputs, train_3_outputs, train_4_outputs, test_outputs, val_outputs]):
    for i in range(0, len(data['input_ids']), batch_size):
        with torch.no_grad():
            batch_outputs = model(**{k: v[i:i+batch_size] for k, v in data.items()})
            outputs.append(batch_outputs.last_hidden_state.mean(dim=1).cpu().numpy())


X_1_train, X_2_train, X_3_train, X_4_train, X_test, X_val = map(torch.tensor, [np.concatenate(train_1_outputs), np.concatenate(train_2_outputs), np.concatenate(train_3_outputs), np.concatenate(train_4_outputs), np.concatenate(test_outputs), np.concatenate(val_outputs)])


Device: cuda


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [9]:

train_model(logistic_classifier, "Bert Logistic Regression", X_1_train, train_1_df['Label'])
train_model(dt_classifier, "Bert Decision Tree", X_2_train, train_2_df['Label'])
train_model(svm_classifier, "Bert SVM", X_3_train, train_3_df['Label'])
train_model(rf_classifier, "Bert Random Forest", X_4_train, train_4_df['Label'])

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Validation Accuracy: 0.9863893561706683


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Test Accuracy: 0.9884389527770981
Validation Accuracy: 0.936779324055666
Test Accuracy: 0.9369035967702471
Validation Accuracy: 0.985471784676556
Test Accuracy: 0.9877660875948128
Validation Accuracy: 0.9692307692307692
Test Accuracy: 0.9744922926351848


In [14]:
# Evaluate ensemble accuracy

def voting(results, dataset):
  # check accuracy for each model
  for key, value in results.items():
    accuracy = accuracy_score(dataset['Label'].fillna(0), value)
    print(key + " Accuracy:", accuracy)

  # voting
  ensemble_predictions = np.sum(list(results.values()), axis=0)
  ensemble_predictions = np.where(ensemble_predictions > (len(results) / 2), 1, 0)

  # find voting accuracy
  accuracy = accuracy_score(dataset['Label'].fillna(0), ensemble_predictions)
  print("Ensemble Accuracy:", accuracy)

  return ensemble_predictions


print("\nEnsemble Voting on Validation dataset:\n")
ensemble_val_predictions = voting(val_results, val_df)

print("\nEnsemble Voting on Test dataset:\n")
ensemble_test_predictions = voting(results, test_df)



Ensemble Voting on Validation dataset:

Roberta Logistic Regression Accuracy: 0.9902431564459397
Roberta Decision Tree Accuracy: 0.9454962532497324
Roberta SVM Accuracy: 0.9847071417647958
Roberta Random Forest Accuracy: 0.9783758984554213
Bert Logistic Regression Accuracy: 0.9863893561706683
Bert Decision Tree Accuracy: 0.936779324055666
Bert SVM Accuracy: 0.985471784676556
Bert Random Forest Accuracy: 0.9692307692307692
Ensemble Accuracy: 0.9833307845236274

Ensemble Voting on Test dataset:

Roberta Logistic Regression Accuracy: 0.9920173721556154
Roberta Decision Tree Accuracy: 0.9465378027893321
Roberta SVM Accuracy: 0.9879190114998777
Roberta Random Forest Accuracy: 0.9811291901149988
Bert Logistic Regression Accuracy: 0.9884389527770981
Bert Decision Tree Accuracy: 0.9369035967702471
Bert SVM Accuracy: 0.9877660875948128
Bert Random Forest Accuracy: 0.9744922926351848
Ensemble Accuracy: 0.98660386591632


In [11]:
import pandas as pd
import tempfile

df = pd.DataFrame.from_dict(results)

with tempfile.TemporaryDirectory() as temp_dir:
    csv_file_path = "/content/drive/MyDrive" + '/predictions.csv'
    df.to_csv(csv_file_path, index=False)
    print("DataFrame saved to", csv_file_path)


DataFrame saved to /content/drive/MyDrive/predictions.csv


In [12]:
predictions_df = pd.DataFrame({
    'Query': test_df['Query'].fillna(0),
    'True_Label': test_df['Label'].fillna(0),
    'Ensemble_Prediction': ensemble_test_predictions
})

predictions_df.to_csv(os.path.join( "/content/drive/MyDrive", "ensemble_predictions.csv"), index=False)
print("Predictions saved successfully.")


Predictions saved successfully.
